In [1]:
#Import required libraries
import torch
import torch.nn as nn
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch_snippets import stems, read
from enel645_group5_functions import *

In [2]:
# Check if GPU is available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print("The device being used is:", device)

The device being used is: cpu


In [3]:
net = UNet().to(device)

PATH = './best_model.pth' # Path to save the best model
net.load_state_dict(torch.load(PATH, map_location=torch.device(device)))

test_ds = SegmentationDataset('test')

test_dl = DataLoader(test_ds, batch_size=8, collate_fn=test_ds.collate_fn)

In [4]:
from torchmetrics import JaccardIndex
jaccard = JaccardIndex(task="multiclass", num_classes=34, average = "weighted")
runningIoU = 0
total_batches = 0

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, data in enumerate(test_dl):
        images, ground_truth_masks = data
        _masks = net(images)
        IoU = jaccard(_masks, ground_truth_masks)
        runningIoU = runningIoU + IoU
        total_batches = i + 1
        print(f"Batch #{total_batches}. RUNNING IoU: {runningIoU / total_batches}")

print(f'IoU of the network on the test images: {runningIoU / total_batches *100} %')

Batch #1. RUNNING IoU: 0.8787014484405518
Batch #2. RUNNING IoU: 0.8510059714317322
Batch #3. RUNNING IoU: 0.8343911170959473
Batch #4. RUNNING IoU: 0.8190805912017822
Batch #5. RUNNING IoU: 0.815911591053009
Batch #6. RUNNING IoU: 0.8244231343269348
Batch #7. RUNNING IoU: 0.8189253211021423
Batch #8. RUNNING IoU: 0.8144890069961548
Batch #9. RUNNING IoU: 0.8137295842170715
Batch #10. RUNNING IoU: 0.8117858171463013
Batch #11. RUNNING IoU: 0.8026285171508789
Batch #12. RUNNING IoU: 0.8021677136421204
Batch #13. RUNNING IoU: 0.798139214515686
Batch #14. RUNNING IoU: 0.8005191683769226
Batch #15. RUNNING IoU: 0.8017549514770508
Batch #16. RUNNING IoU: 0.8017579317092896
Batch #17. RUNNING IoU: 0.8045522570610046
Batch #18. RUNNING IoU: 0.8025670647621155
Batch #19. RUNNING IoU: 0.8026304244995117
Batch #20. RUNNING IoU: 0.8039429783821106
Batch #21. RUNNING IoU: 0.805064857006073
Batch #22. RUNNING IoU: 0.8062458038330078
Batch #23. RUNNING IoU: 0.8068971037864685
Batch #24. RUNNING IoU: